# Aufgabe 3 – Fragestellungen und Zielvariablen begrenzen die Methodenauswahl

# Aufgabe 3a)

**Aufgabenstellung**

Untersuchen Sie die Art der Variablen und die Verteilung der Zielvariablen. Gegebenenfalls ist eine Datenvorverarbeitung nötig. Nennen Sie die möglichen Methoden. Wählen Sie eine Methode aus und begründen Sie Ihre Auswahl. Bitte separieren Sie die Datensätze mit 80% : 20% = Training : Test und führen Sie die Methode ohne Parameteroptimierung aus. Vergleichen Sie die Ergebnisse mit denen von vorgegebenen Methode(n).

Der Datensatz `employment_08_09.xlsx` beinhaltet die sozioökonomischen Daten der Arbeitskräfte in den USA im April 2008 und Angaben, ob sie im April 2009 weiterhin angestellt sind. Alle Befragten waren im April 2008 angestellt. Sagen Sie basierend auf den 2008er Informationen vorher, welche Arbeitskraft 2009 arbeitslos wird. Haben ältere Arbeitskräfte ein höheres Risiko für Arbeitslosigkeit während der Finanzkrise 2008-2009?

**Zusammenfassung der Ergebnisse**

Der Datensatz ist stark unbalanciert, was das Training erschwert. Nach eingehender Analyse ist das Ergebnis, dass ältere Arbeitskräfte kein höheres Risiko für Arbeitslosigkeit während der Finanzkrise haben.

## Modul-Importe und Einstellungen

In diesem Abschnitt werden die benötigten Python-Module mithilfe von `import` importiert.

In [ ]:
# Set Module Path
import sys
sys.path.append('./module')

# Own Modules
import one_hot

# Common Libraries
import pandas as pd
import seaborn as sns
import numpy as np

# SimpleImputer from sklearn (Can replace missing values)
from sklearn.impute import SimpleImputer

# OneHotEncoder from sklearn (Can do one-hot encoding)
from sklearn.preprocessing import OneHotEncoder

# Metrics
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix

# Classifiers
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

## Einlesen des Datensatzes

In diesem Abschnitt wird der Datensatz als Excel-Datei (`.xlsx`) mithilfe von `pandas` eingelesen.

In [ ]:
# Load dataset from Excel file
data = pd.read_excel('./data/employment_08_09.xlsx')

## Untersuchung der Merkmale

In der [Beschreibung des Datensatzes](./data/employment_08_09_description.pdf) wird unter anderem folgende Information gegeben:

"These data file contains data on 5412 workers who were survey in the April 2008 Current Population Survey and reported that they were employed. The data file contains their employment status in April 2009, one year later, along with some additional variables."

Zunächst soll überprüft werden, ob der Datensatz vollständig geladen wurde.

In [ ]:
# Get info on Datasat
data.info()

Anmerkungen: 
- Alle 5412 Datensätze wurden geladen und 
- Es gibt nur in der Spalte `earnwke` fehlende Werte

#### Beschreibung und Einordnung der Merkamle

Die [Beschreibung des Datensatzes](./data/employment_08_09_description.pdf) enhält zudem Informationen zur Bedeutung der Spalten. In der folgenden Zelle ist diese mit einer Einordnung der Merkmale angereichert.

- Beschäftigungsstatus des Arbeiters im Jahr 2009 (_Zielvariable_): Kategorisch-Nominal
    - `employed`: Integer (= 1, wenn im Jahr 2009 noch angestellt)
    - `unemployed`: Integer (= 1, wenn im Jahr 2009 nicht mehr angestellt)
    
Informationen über den Arbeiter im Jahr 2008:

- Alter: Numerisch-Kontinuierlich
    - `age`: Integer

- Geschlecht: Kategorisch-Nominal
    - `female`: Integer (= 1, wenn weiblich)

- Familienstand: Kategorisch-Nominal
    - `married`: Integer (= 1, wenn Verheiratet)

- Selbst-Identifizierte Rasse, Kategorisch-Nominal
    - `race`: Integer (weiß=1, schwarz=2, andere=3)

- Gewerkschaftszugehörigkeit: Kategorisch-Nominal
    - `union`: Integer (= 1, wenn Gewerkschaftsmitglied)

- Bundesstaats-Angehörigkeit: Kategorisch-Nominal
    - `ne_states`: Integer (= 1, wenn von Nordost-Staat)
    - `so_states`: Integer (= 1, wenn von Südstaat)
    - `ce_states`: Integer (= 1, wenn von zentralem Staat)
    - `we_states`: Integer (= 1, wenn von Weststaat)

- Angestelltenverhältnis: Kategorisch-Nominal
    - `private`: Integer (= 1, wenn angestellt in privatem Betrieb)
    - `government`: Integer (= 1, wenn angestellt von der Regierung)
    - `self`: Integer (= 1, wenn selbstständig)

- Höchster Abschluss: Kategorisch-Ordinal
    - `educ_lths`: Integer (= 1, wenn höchster Abschluss niedriger als High-School)
    - `educ_hs`: Integer (= 1, wenn höchster Abschluss ist High-School)
    - `educ_somecol`: Integer (= 1, wenn höchster Abschluss ist College)
    - `educ_aa`: Integer (= 1, wenn höchster Abschluss ist AA)
    - `educ_ba`: Integer (= 1, wenn höchster Abschluss ist BA oder BS)
    - `educ_adv`: Integer (= 1, wenn höchster Abschluss fortgeschritten)

- Durschnittliches wöchentliches Einkommen: Numerisch-Kontinuierlich
    - `earnwke`: Float




## Verteilung der Zielvariable

###### Analyse

Bevor Aussagen über die Verteilung der Zielvariable gemacht werden, soll analysiert überprüft werden, ob inkonsistente oder unvollständige Datensätze bezüglich der Zielvariable vorliegen.

In [ ]:
# Number of objects with contradiction (employed and unemployed)
data[(data['employed']==1) & (data['unemployed']==1)].count()[0]

Es gibt keine Datenobjekte, die eine ungültige Beschäftigungsinformation enthalten, bei welcher der Arbeiter gleichzeitig beschäftigt und nicht beschäftigt ist.

In [ ]:
# Are there objects with no information (not employed and not unemployed)
data[(data['employed']==0) & (data['unemployed']==0)].count()[0]

Es gibt 435 Datenobjekte, die keine Information über den Beschäftigungsstatus 2009 enthalten. Diese sollten bei der Datenvorverarbeitung entfernt werden, da sie zum Trainieren / Testen nicht verwendet werden können.

###### Graphische Darstellung

Ermitteln der Daten für die graphische Darstellung.

In [ ]:
# Preparing values for the barplot
employed = data[data['employed']==1].count()[0]
unemployed = data[data['unemployed']==1].count()[0]
no_information = data[(data['employed']==0) & (data['unemployed']==0)].count()[0]

Graphische Darstellung der Verteilung mithilfe von `seaborn` und einem neuen `dataframe` zur Beschriftung der Achsen.

In [ ]:
# Instanciating a new dataframe to have a nice barplot with descriptions
plot_data = pd.DataFrame({'Employment Status 2009': ['employed', 'unemployed', 'no_information'], 
                          'Number of Employees': [employed, unemployed, no_information]})
# Draw the barplot with seaborn
ax = sns.barplot(x='Employment Status 2009', y='Number of Employees', data=plot_data, ci=None)

Erkenntnis: Die Zielvariable ist sehr ungleich verteilt.

Anmerkung: Datenobjekte ohne Beschäftigungsstatus 2009 werden bei der Datenvorverarbeitung entfernt, da sie zum Trainieren / Testen nicht verwendet werden können.

###### Ermittlung der prozentualen Verteilung der Zielvariable

In [ ]:
# Ratio of employed workers
employed / (employed + unemployed)

Erkenntnis: Nur ungefähr 5% der Arbeiter hatten im Jahr 2008 keine Beschäftigung mehr. Hierbei handelt es sich also um einen unbalancierten Datensatz.

## Datenvorverarbeitung

In diesem Abschnitt werden die Daten in Hinblick auf das Training der Modelle vorverarbeitet.

###### 1. Entfernen der Datenobjekte ohne Beschäftigungsinformation 2009

In [ ]:
# Determine rows with no information about Employment Status 2009
rows_no_onformation = data[(data['employed']==0) & (data['unemployed']==0)]

# Drop those rows with no information
data.drop(rows_no_onformation.index, inplace=True)

Neue Größe des Datensatzes überprüfen

In [ ]:
# New size of the dataframe
data.shape

###### 2. Trennen der unabhängigen Daten (`X`) von den abhängigen Daten (Zielvariable `y`)

Mermale `X` sind alle Daten außer die Spalten der Zielvariable:

In [ ]:
# Don't include Employment Information 2009 in the features
X = data.drop(['employed', 'unemployed'], axis=1)

Überprüfen, ob die Spalten der Zielvariable entfernt wurde.

In [ ]:
# Check if Employment Information was removed by printing out the feature columns
X.columns

Erstellen der Zielvaribale `y`.

In [ ]:
# Output variable is the Employment Information 2009
y = data[['employed', 'unemployed']]

###### 3. Fehlende Daten in der Spalte `earnwke` ergänzen

Fehlende Daten sollen durch das durchschnittliche Einkommen ersetzt werden.  

Anmerkung: Dies ist wahrscheinlich nicht die beste Option, da vor allem bei Selbstständige Arbeitern diese Angabe fehlt.

In [ ]:
# Instanciate SimpleImputer with np.nan and strategy 'mean' (replaces missing values with mean)
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')

# Fit the Imputer on the data and transform it
X[["earnwke"]] = imputer.fit_transform(X[["earnwke"]].to_numpy())

Überprüfen, ob fehlende Werte ersetzt wurden.

In [ ]:
# Check if missing values are replaced
X[["earnwke"]].info()

Das Ersetzen der Werte war erfolgreich.

###### 4. Überprüfen der One-Hot encodierten Daten

Bei One-Hot enkodierten Daten können Inkonsistenzen auftreten. Das Modul `one_hot.py` enthält eine Funktion namens `is_one_hot_encoded`, die solche Inkonsistenzen erkennen kann. Die folgende Zelle überprüft die one-hot-enkodierten Informationen mithilfe dieser Funkion auf Inkonsistenzen.

In [ ]:
# Check columns for correct one-hot encoding
[ one_hot.is_one_hot_encoded(X[['ne_states', 'so_states', 'ce_states', 'we_states']]),
  one_hot.is_one_hot_encoded(X[['government', 'private', 'self']]),
  one_hot.is_one_hot_encoded(X[['educ_lths', 'educ_hs', 'educ_somecol', 'educ_aa', 'educ_bac', 'educ_adv']])]

Erkenntnis: Die Informatioen über _Bundesstaats-Angehörigkeit_, _Beschäftigungsverhältnis_ und _höchster Abschluss_ sind korrekt und vollständig One-Hot-Enkodiert. Es gibt keine Widersprüchlichkeiten oder fehlende Informationen.

Da das Merkmal _höchster Abschluss_ kategorisch-ordinal ist, kann man dieses in einer Variable kodieren, was in der folgenden Zelle geschieht. Dafür wird die Funktion `one_hot_to_ordinal`verwendet, die ebenfalls im Modul `one_hot.py` enthalten ist.

Die anderen one-hot-enkodierten kategorischen Merkmale sollten nicht umgewandelt werden, da sie nicht ordinal sind.

In [ ]:
# Create the 'educ' column on basis of the 'educ_' columns
X['educ'] = one_hot.one_hot_to_ordinal(X[['educ_lths', 'educ_hs', 'educ_somecol', 'educ_aa', 'educ_bac', 'educ_adv']])
X = X.drop(['educ_lths', 'educ_hs', 'educ_somecol', 'educ_aa', 'educ_bac', 'educ_adv'], axis=1)

# X['employment'] = one_hot_to_ordinal(X[['self', 'government', 'private']])
# X = X.drop(['self', 'government', 'private'], axis=1)

# X['state'] = one_hot_to_ordinal(X[['ne_states', 'so_states', 'ce_states', 'we_states']])
# X = X.drop(['ne_states', 'so_states', 'ce_states', 'we_states'], axis=1)

Überprüfen, ob die Spalte `educ` hinzugefügt wurde und die anderen Spalten entfernt wurden.

In [ ]:
# Check column names
X.columns

Überprüfen, ob die Zielvariable ohne Inkonsistenzen one-hot encodiert ist:

In [ ]:
# Check if output variable is one-hot encoded
one_hot.is_one_hot_encoded(y[['employed', 'unemployed']])

Erkenntnis: Das Entfernen der Datenobjekte mit fehlenden Informationen über das _Beschäftigungsstatus 2009_ (Schritt 1) hat funktioniert.

Die Spalte `unemployed` kann jetzt entfertn werden, da sie gegenüber der Spalte `employed` keine neuen Informationen enthält:

In [ ]:
# Removing the column `unemployed` from y
y = y.drop(['unemployed'], axis=1)

# Check if it worked
y.columns

Das Entfernen der Spalte `unemployed` war erfolgreich.

###### 5. One-Hot Enkodierung

Die Information über die selbstidentifizierte Rassenzugehörigkeit ist in den Daten numerisch in einer Spalte numerisch enkodiert. Da es sich jedoch um ein Attribut ohne Ordnung handelt (nominal), sollte diese Information one-hot-enkodiert werden. Dies geschieht in der folgenden Zelle:

In [ ]:
# Instanciate OneHotEncoder with determined indices
onehotencoder = OneHotEncoder(categories = 'auto')

# Compute one-hot encoded columns
race_one_hot = onehotencoder.fit_transform(X[['race']]).toarray()

# Remove original column
X = X.drop(['race'], axis=1)

# Add one-hot encoded columns
X['race_white'], X['race_black'], X['race_other'] = np.split(race_one_hot, indices_or_sections=3, axis=1)

Überprüfuen, ob die entsprechenden Spalten hinzugefügt wurden:

In [ ]:
# Check if it worked
X.columns

Erkenntnis: Die Spalten wurden erfolgreich hinzugefügt.

###### 6. Auteilung in Training-  und Test-Daten

In [ ]:
# Import train_test_split from sklearn (Can do splitting)
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

###### 7. Feature-Scaling

In [ ]:
# Import StandardScaler from sklearn (Can do feature scaling)
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

In [ ]:
sc_X = RobustScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

## Training und Modellauswahl

###### Mögliche Methoden

Es handelt sich bei der Entscheidung, ob ein Beschäftigter aus dem Jahr 2008 auch im Jahr 2009 noch beschäftigt ist, um eine Klassifikationsaufgabe. 

Zudem soll bestimmt werden, ob ältere Arbeitskräfte ein ein höheres Risiko für Arbeitslosigkeit im Jahr 2009 haben. 

Deshalb sollte eine Klassifikations-Methode gewählt werden, bei der nach dem Training bestimmt werden kann, welche Features besonders großen Einfluss auf die Klassifikation haben.

Mögliche Methoden hierfür sind _Lineare Regression_, _Logistische Regression_ und _Entscheidungsbäume_.

Aufgrund der Tatsache, dass es sich um einen unbalancierten Datensatz handelt, wird neben der Metrik "accuracy_score" zusätzlich die Metrik "balanced_accuracy_score" verwendet, welche den durschnittlichen Recall auf allen Klassen berechnet.

Siehe auch: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html

###### Baseline: Häufigste Klasse

Als erste Baseline soll das Vorschlagen der am häufigsten Auftretenden Klasse untersucht werden.

In [ ]:
# Accuracy on Training Data
accuracy_score(y_train, np.ones_like(y_train))

In [ ]:
# Accuracy on Test Data
accuracy_score(y_test, np.ones_like(y_test))

In [ ]:
balanced_accuracy_score(y_test, np.ones_like(y_test))

Wie erwartet. erreicht man mit der Vorhersage der häufigsten Klasse eine Accuracy von ca 95% und eine Balanced Accuracy von 0,5.

###### Baseline: Lineare Regression

Als zweite Baseline soll ein Klassifikator basierend auf linearer Regression untersucht werden.

In [ ]:
# Train linear Regression
model_lir = LinearRegression().fit(X_train, y_train['employed'])

In [ ]:
# Define Classification prediction function for linear Regression
def predict_lir(X):
    predictions = model_lir.predict(X)
    predictions[predictions>=0.5] = 1
    predictions[predictions<0.5] = 0
    return predictions

In [ ]:
# Accuracy Scores
accuracy_score(y_train, predict_lir(X_train))

In [ ]:
# Accuracy on Test Data
accuracy_score(y_test, predict_lir(X_test))

In [ ]:
# Calculate Balanced Accuracy
balanced_accuracy_score(y_test, predict_lir(X_test))

In [ ]:
# Print confusion Matrix
print("tn: {}, fp: {}\nfn: {}, tp: {}".format(*confusion_matrix(y_test, predictions).ravel()))

Die Metriken zeigen für den Klassifikator basierend auf linearer Regression sind die gleichen Ergebnisse wie für den Klassifikator, der Objekte immer der häufigsten Klasse zuordnet.

Anhand der Confusion Matrix ist zu sehen, dass auch dieser Klassifikator die Objekte immer der häufigsten Klasse zuzuordnet.

###### Logistische Regression 

In den folgenden Zellen wird ein Klassifikator trainiert, der auf logistischer Regression basiert. Es wird der Parameter `class_weight = balanced` mitgegeben, um die unterrepräsentierte Klasse (`unemployed`) höher zu gewichten, da es sich um einen unbalancierten Datensatz handelt.

In [ ]:
# Train Logistic regression classifier with balanced class weight
model_lr = LogisticRegression(solver='lbfgs', class_weight = 'balanced').fit(X_train, y_train['employed'])

In [ ]:
# Accuracy on Training Data
model_lr.score(X_train, y_train)

In [ ]:
# Accuracy on Test Data
model_lr.score(X_test, y_test)

In [ ]:
# Calculate Balanced Accuracy
balanced_accuracy_score(y_test, model_lr.predict(X_test))

In [ ]:
# Print confusion Matrix
print("tn: {}, fp: {}\nfn: {}, tp: {}".format(*confusion_matrix(y_test, model_lr.predict(X_test)).ravel().ravel()))

Auch wenn die Accuracy bei der logistischen Regression mit `class_weight = balanced` schlechter ist, zeigt die Metrik "Balanced Accuracy" Ergebnisse als die beiden Baseline-Methoden.

An der Konfusionsmatrix ist zu erkennen, dass dieser Klassifikator nicht alle Daten der häufigsten Klasse zuordnet.

###### Entscheidungsbaum (Decision Tree)

Als weitere mögliche Klassifikations-Methode soll ein Entscheidungsbaum untersucht werden, bei dem ebenfalls der Parameter `class_weight = 'balanced'` gesetzt wird, da es ich um einen unbalanvierten Datensatz handelt.

In [ ]:
# Train decision tree classifier with balanced class weight
model_dt = DecisionTreeClassifier(class_weight = 'balanced').fit(X_train, y_train)

In [ ]:
# Accuracy on Training Data
model_dt.score(X_train, y_train)

In [ ]:
# Accuracy on Test Data
model_dt.score(X_test, y_test)

In [ ]:
# Calculate Balanced Accuracy
balanced_accuracy_score(y_test, model_dt.predict(X_test))

Die Metriken zeigen für den Klassifikator basierend auf einem Entscheidungsbaum, dass er zwar sehr gut gelernt hat, die Trainingsdaten zu klassifizieren, jedoch auf den Testdaten eine schlechtere Accuracy hat. Zudem ist die Balanced Accuracy auf den Testdaten sogar leicht schlechter als bei der Baseline.

## Schlussfolgerungen

###### Haben ältere Arbeitskräfte ein höheres Risiko für Arbeitslosigkeit während der Finanzkrise 2008-2009?

Um diese Frage zu beantworten, wird im Folgenden das beste Modell verwendet, um zu bestimmen, wie groß der Einfluss der einzelnen Merkmale auf die Zielvariable ist. Dabei handelt es sich um einen Klassifikator basierend auf "Logistischer Regression".

Dieser Klassifikator trainiert Koeffizienten für die verschiedenen Merkmale. Da die Merkmale im letzten Schritt der Datenvorverarbeitung normalisiert wurden, können die trainierten Koeffizienten verwendet werden, um zu bestimmen, wie groß der Einfluss des jeweililgen Merkmals auf die Zielvariable ist. Dazu muss der absolute Wert des Koeffizienten betrachtet werden.

Siehe auch: https://blog.minitab.com/blog/adventures-in-statistics-2/how-to-identify-the-most-important-predictor-variables-in-regression-models

In der folgenden Liste sind  die Merkmale nach ihrem Einfluss auf die Zielvariable sortiert:

In [ ]:
# Feature importance according to Logistic Regression Classifier
sorted(zip(X.columns.tolist(), model_lr.coef_[0]), key=lambda x: abs(x[1]), reverse = True)

Laut dieser Liste hat das Merkmal `age`  auf die Arbeitslosigkeit nur den zehnt-größten Einfluss von insgesamt 16 Variablen. Dessen Koeffizient hat einen Betrag von `0.13`, während der Betrag des größten Koeffizients (`private`) `0.94` ist.

Die Frage, ob ältere Arbeitskräfte ein höheres Risiko für Arbeitslosigkeit haben, ist daher mit "Nein" zu beantworten.